In [4]:
from typing import List

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table

import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff

from flask_cors import CORS

import pandas as pd

import os
import requests
import io

In [5]:
url='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master/GUI_in.csv'
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')

df['Dprio']=df['Dprio'].astype('float')
df['NumPax']=df['NumPax'].astype('float')
df['Ideal_lower']=df['Ideal_lower'].astype('float')
df['Ideal_average_downweighted']=df['Ideal_average_downweighted'].astype('float')
df['Ideal_average_full']=df['Ideal_average_full'].astype('float')
df['Ideal_upper']=df['Ideal_upper'].astype('float')

dep_dates = df['DepDate'].unique()
dtime = df['dtime'].unique()

# Function for conditional coloring

In [6]:
COLORS = [
    {
        'background': '#FF0000',
        'text': 'black'
    },
    {
        'background': '#5DFC0A',
        'text': 'black'
    },
]

def cell_style(value1, value2, value3):

    # style = {}
    relative_value1 = value1 - value2
    relative_value2 = value1 - value3

    if relative_value1 < 0 or relative_value2 > 0:
            style = {
                'backgroundColor': COLORS[0]['background'],
                'color': COLORS[0]['text']
            }
    else:
            style = {
                'backgroundColor': COLORS[1]['background'],
                'color': COLORS[1]['text']
            }
    return style

# Function for table creation

In [7]:
def generate_table(dataframe, max_rows=50):
    rows = []
    for i in range(min(len(dataframe), max_rows)):
        row = []
        for col in dataframe.columns:
            if col == 'NumPax':
                value1 = dataframe.iloc[i][col]
                value2 = dataframe.iloc[i]['Ideal_lower']
                value3 = dataframe.iloc[i]['Ideal_upper']
                style = cell_style(value1, value2, value3)
                row.append(html.Td(value1, style=style))
            else:
                value = dataframe.iloc[i][col]
                row.append(html.Td(value))
        rows.append(html.Tr(row))

    return html.Table(
            # Header
            [html.Tr([html.Th(col) for col in dataframe.columns])] +

            # Body
            rows)

# Import external stylesheet

In [8]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Core code

In [ ]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    # Header
    html.H4(children='Booking curve steering system'),
    # Dropdown for departure dates
    dcc.Dropdown(id = 'dropdown_depdates',
    options=[{
    'label': i,
    'value': i
    } for i in dep_dates],
    value='All Departure Dates'),
    # Dropdown for departure times
    dcc.Dropdown(id = 'dropdown_dtime',
    options=[{
    'label': i,
    'value': i
    } for i in dtime],
    value='All Departure Times'),    
    html.Div(id='table-container'),
])

@app.callback(Output('table-container', 'children'), [Input('dropdown_depdates', 'value'), Input('dropdown_dtime', 'value')])
def update_table(value_depdate, value_dtime):
    dff = df[(df['DepDate'] == value_depdates)&(df['DepDate'] == value_dtime)] # update with your own logic
    return generate_table(dff)

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2019 10:56:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:56:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2019 10:56:25] "GET /_dash-dependencies HTTP/1.1" 200 -
[2019-04-08 10:56:25,707] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise va

127.0.0.1 - - [08/Apr/2019 10:56:35] "POST /_dash-update-component HTTP/1.1" 500 -
[2019-04-08 10:56:38,263] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\Users\dkoch\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\dkoch\A